# Imports

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import permutations, product
from collections import defaultdict
from time import strftime, gmtime
import random
from random import sample

import string
# import pysign.utils
# from pysign.infos import location_movement

# Params

In [3]:
subject = 23
video_dir = "robot_stimuli"
cdir = os.getcwd()
subject_dir = f"{cdir}\\data\\subject_{subject}"

# Signs

In [4]:
def config_to_name(configuration):
    return f"{configuration['laterality']}_{configuration['localisation']}_{configuration['movement']}_{configuration['direction']}"

def name_to_config(fname):
    return dict(zip(['laterality', 'localisation', 'movement', 'direction'], fname.split('_')))

In [5]:
configurations = {
    'laterality' : ['L', 'R'],
    'localisation' : ['Head', 'Arm', 'Torso', 'Base'],
    'movement' : ['X', 'Y', 'Z'],
    'direction' : ['For', 'Back']
    }

all_configurations = list(product(*configurations.values()))

In [6]:
row_list = []

all_configurations = list(product(*configurations.values()))

for configuration in all_configurations:
    row = dict(zip(configurations.keys(), configuration))
    row_list.append(row)

movement_df = pd.DataFrame(row_list)

movement_df["fname"] = movement_df.apply(config_to_name, axis=1)

movement_df

,laterality,localisation,movement,direction,fname
0,L,Head,X,For,L_Head_X_For
1,L,Head,X,Back,L_Head_X_Back
2,L,Head,Y,For,L_Head_Y_For
3,L,Head,Y,Back,L_Head_Y_Back
4,L,Head,Z,For,L_Head_Z_For
5,L,Head,Z,Back,L_Head_Z_Back
6,L,Arm,X,For,L_Arm_X_For
7,L,Arm,X,Back,L_Arm_X_Back
8,L,Arm,Y,For,L_Arm_Y_For
9,L,Arm,Y,Back,L_Arm_Y_Back


# Sternberg

## Params

In [8]:
#Global params
video_dir = "robot_stimuli"
subject = 23
sequence_sizes = [2,4,6]
# sequence_sizes = [3,4,5]
true_rate = 0.5

#Main task
n_block = 3
n_trial = 10


#Training
training_block = 1
training_trial = 3

## Time estimation

In [9]:
#Global times
block_intro_duration = 60
inter_trial = 1

#Letter times
letter_stimuli_duration = 2
letter_probe_duration = 2
letter_maintenance_period = 1

#Robot times
robot_stimuli_duration = 3.5
robot_probe_duration = 3.5
robot_maintenance_period = 1

robot_trial_durations = {sequence_size : inter_trial+(robot_stimuli_duration*sequence_size)+robot_maintenance_period+robot_probe_duration for sequence_size in sequence_sizes}
letter_trial_durations = {sequence_size : inter_trial+(letter_stimuli_duration*sequence_size)+letter_maintenance_period+letter_probe_duration for sequence_size in sequence_sizes}

total_trial_durations = pd.DataFrame(data = {"sequence_sizes": sequence_sizes})
total_trial_durations['robot duration'] = total_trial_durations["sequence_sizes"].apply(lambda x : robot_trial_durations[x])
total_trial_durations['letter duration'] = total_trial_durations["sequence_sizes"].apply(lambda x : letter_trial_durations[x])

total_robot_duration = sum([(trial_duration*n_trial)+block_intro_duration for trial_duration in robot_trial_durations.values()]*len(sequence_sizes))
total_letter_duration = sum([(trial_duration*n_trial)+block_intro_duration for trial_duration in letter_trial_durations.values()]*len(sequence_sizes))

print(f"A total of {n_block*n_trial} trials for each condition ({len(sequence_sizes)}) and each stimuli type (2)")
display(total_trial_durations.set_index("sequence_sizes").T)
print(f"Estimated robot duration {strftime('%X', gmtime(total_robot_duration-(len(sequence_sizes)*n_block*block_intro_duration)))} / {strftime('%X', gmtime(total_robot_duration))}")
print(f"Estimated letter duration {strftime('%X', gmtime(total_letter_duration-(len(sequence_sizes)*n_block*block_intro_duration)))} / {strftime('%X', gmtime(total_letter_duration))}")

print(f"Estimated total duration {strftime('%X', gmtime(total_robot_duration+total_letter_duration-(2*len(sequence_sizes)*n_block*block_intro_duration)))} / {strftime('%X', gmtime(total_robot_duration+total_letter_duration))}")

A total of 30 trials for each condition (3) and each stimuli type (2)


sequence_sizes,2,4,6
robot duration,12.5,19.5,26.5
letter duration,8.0,12.0,16.0


Estimated robot duration 00:29:15 / 00:38:15
Estimated letter duration 00:18:00 / 00:27:00
Estimated total duration 00:47:15 / 01:05:15


## Utils

In [10]:
def get_block_df(condition_df_list):
    condition_sequence =  [df['condition'].unique()[0] for df in condition_df_list]

    counter_dict = defaultdict(int)
    block_sequence = []
    for condition in condition_sequence:
        counter_dict[condition] += 1
        block_sequence.append(counter_dict[condition])
    
    for i, df in enumerate(condition_df_list):
        df['block'] = block_sequence[i]
        df['is_block_beginning'] = [True if i==0 else False for i in range(len(df))]

    return pd.concat(condition_df_list).reset_index(drop=True)

In [11]:
def get_sternberg_df(sequence_size, true_rate, n_trial, items):
    if int(1/true_rate) != 1/true_rate:
        raise(Warning('Warning : true_rate must be a unit fraction'))

    sequences = []
    for i in range(n_trial):
        trial = pd.Series(dtype='object')

        sample = random.sample(items, k=sequence_size+1)
        trial['sequence'], absent_target = sample[:-1], sample[-1]

        trial['is_true'] = i%(1/true_rate)==0

        if trial['is_true'] :
            present_target = random.choice(trial['sequence'])
            trial['target'] = present_target
        else:
            trial['target'] = absent_target

        sequences.append(trial)
    
    sequence_df = pd.DataFrame(sequences)
    sequence_df['condition'] = sequence_size
    sequence_df = sequence_df.sample(frac=1).reset_index(drop=True)

    return sequence_df

# sequence_size = 5
# stimuli_type = 'letter'

# true_rate = 0.5

# item_dict = {
#     'robot':movement_df['fname'].to_list(),
#     'letter':[elem for elem in string.ascii_uppercase if elem not in 'AEIOUY']
#     }


# get_sternberg_df(sequence_size, true_rate, n_trial, items=item_dict['robot'])


## Computation

In [12]:
def get_conditions(subject, n_trial, sequence_sizes, true_rate, n_block, stimuli_order, item_dict, video_dir, verbose=False):
    df_list = []

    for block in range(n_block):
        for stimuli_type in stimuli_order:
            for sequence_size in sample(sequence_sizes, len(sequence_sizes)):
                if verbose:
                    print(f"{sequence_size} item sequence, block {block}, {stimuli_type} stimuli")
                df = get_sternberg_df(sequence_size, true_rate, n_trial, items=item_dict[stimuli_type])

                #Metadata
                df['stimuli'] = stimuli_type
                df['block'] = block+1
                df['is_block_beginning'] = [True if i==0 else False for i in range(len(df))]
                df['is_block_ending'] = [True if i==(n_trial-1) else False for i in range(len(df))]

                df_list.append(df)
                
                if verbose:
                    print(df['is_true'].value_counts().to_dict())
                    print('-------------------------------------------------------------')
    
    sternberg_df = get_block_df(df_list)
    sternberg_df['sequence_fnames'] = sternberg_df['sequence'].apply(lambda x : [f"{video_dir}\\{elem}.mp4" if len(elem)>1 else f"{video_dir}\\\\filler.mp4" for elem in x])
    sternberg_df['target_fname'] = sternberg_df['target'].apply(lambda x : f"{video_dir}\\{x}.mp4" if len(x)>1 else f"{video_dir}\\\\filler.mp4")
    sternberg_df['subject'] = subject

    return sternberg_df

In [13]:
def get_training(subject, n_trial, sequence_sizes, true_rate, n_block, stimuli_order, item_dict, video_dir, verbose=False):
    df = get_conditions(subject=subject, n_trial=n_trial, sequence_sizes=sequence_sizes, true_rate=true_rate, n_block=n_block, 
                                    stimuli_order=stimuli_order, item_dict=item_dict, video_dir=video_dir, verbose=verbose)
    
    df['stimuli'] = pd.Categorical(df['stimuli'], categories=['letter', 'robot'], ordered=True)
    df = df.sort_values(by=['stimuli', 'condition'])
    
    return df

In [14]:
item_dict = {
    'robot':movement_df['fname'].to_list(),
    'letter':[elem for elem in string.ascii_uppercase if elem not in 'AEIOUY']
    }

stimuli_order = ['letter', 'robot']

In [19]:
sternberg_df = get_conditions(subject=subject, n_trial=n_trial, sequence_sizes=sequence_sizes, true_rate=true_rate, n_block=n_block, 
stimuli_order=stimuli_order, item_dict=item_dict, video_dir=video_dir, verbose=True)

3 item sequence, block 0, letter stimuli
{False: 5, True: 5}
-------------------------------------------------------------
4 item sequence, block 0, letter stimuli
{True: 5, False: 5}
-------------------------------------------------------------
5 item sequence, block 0, letter stimuli
{True: 5, False: 5}
-------------------------------------------------------------
4 item sequence, block 0, robot stimuli
{True: 5, False: 5}
-------------------------------------------------------------
3 item sequence, block 0, robot stimuli
{True: 5, False: 5}
-------------------------------------------------------------
5 item sequence, block 0, robot stimuli
{False: 5, True: 5}
-------------------------------------------------------------
4 item sequence, block 1, letter stimuli
{True: 5, False: 5}
-------------------------------------------------------------
5 item sequence, block 1, letter stimuli
{True: 5, False: 5}
-------------------------------------------------------------
3 item sequence, bl

In [15]:
sternberg_training_df = get_training(subject=subject, n_trial=training_trial, sequence_sizes=sequence_sizes, true_rate=true_rate, n_block=training_block, 
stimuli_order=stimuli_order, item_dict=item_dict, video_dir=video_dir, verbose=True)

4 item sequence, block 0, letter stimuli
{True: 2, False: 1}
-------------------------------------------------------------
6 item sequence, block 0, letter stimuli
{True: 2, False: 1}
-------------------------------------------------------------
2 item sequence, block 0, letter stimuli
{True: 2, False: 1}
-------------------------------------------------------------
2 item sequence, block 0, robot stimuli
{True: 2, False: 1}
-------------------------------------------------------------
4 item sequence, block 0, robot stimuli
{True: 2, False: 1}
-------------------------------------------------------------
6 item sequence, block 0, robot stimuli
{True: 2, False: 1}
-------------------------------------------------------------


## Saving

In [16]:
def safe_save(df, save_dir, fname, index=False):
    fname_versions = [elem for elem in os.listdir(save_dir) if elem.split('old_')[-1]==fname]
    sorted_fname_versions = sorted(fname_versions, key=len, reverse=True)

    for target_name in sorted_fname_versions:
        os.rename(f"{save_dir}\\{target_name}", f"{save_dir}\\old_{target_name}")

    
    df.to_csv(f"{save_dir}\\{fname}", index=index)

In [22]:
os.makedirs(subject_dir, exist_ok = True)

In [23]:
fname = f"conditions_sternberg.csv"
safe_save(sternberg_df, subject_dir, fname, index=False)

In [24]:
fname = f"conditions_sternberg_short.csv"
safe_save(sternberg_training_df.head(training_trial), subject_dir, fname, index=False)

In [17]:
fname = f"conditions_sternberg_training.csv"
safe_save(sternberg_training_df, subject_dir, fname, index=False)

## Multi-subject

In [30]:
subjects = range(24)

mode = 'hard'

item_dict = {
    'robot':movement_df['fname'].to_list(),
    'letter':[elem for elem in string.ascii_uppercase if elem not in 'AEIOUY']
    }

for subject in subjects:
    subject_dir = f"{cdir}\\data\\subject_{subject}"

    if subject%2==0:
        stimuli_order = ['letter', 'robot']

    elif subject%2==1:
        stimuli_order = ['robot', 'letter']

    sternberg_df = get_conditions(subject=subject, n_trial=n_trial, sequence_sizes=sequence_sizes, true_rate=true_rate, n_block=n_block, 
                                stimuli_order=stimuli_order, item_dict=item_dict, video_dir=video_dir, verbose=False)
    sternberg_training_df = get_training(subject=subject, n_trial=training_trial, sequence_sizes=sequence_sizes, true_rate=true_rate, n_block=training_block, 
                                        stimuli_order=stimuli_order, item_dict=item_dict, video_dir=video_dir, verbose=False)

    os.makedirs(subject_dir, exist_ok = True)

    if mode == 'hard':
        sternberg_df.to_csv(f"{subject_dir}\\conditions_sternberg.csv", index=False)
        sternberg_training_df.head(training_trial).to_csv(f"{subject_dir}\\conditions_sternberg_short.csv", index=False)
        sternberg_training_df.to_csv(f"{subject_dir}\\conditions_sternberg_training.csv", index=False)
    
    elif mode == 'safe':
        safe_save(sternberg_df, subject_dir, "conditions_sternberg.csv", index=False)
        safe_save(sternberg_training_df.head(), subject_dir, "conditions_sternberg_short.csv", index=False)
        safe_save(sternberg_training_df, subject_dir, "conditions_sternberg_training.csv", index=False)
